In [10]:
import json
import sys
import importlib.util
from pathlib import Path

In [2]:
data = {"question": "Find the area of a triangle with a base of 10 units and height of 5 units.", "function": {"name": "calculate_triangle_area", "description": "Calculate the area of a triangle given its base and height.", "parameters": {"type": "dict", "properties": {"base": {"type": "integer", "description": "The base of the triangle."}, "height": {"type": "integer", "description": "The height of the triangle."}, "unit": {"type": "string", "description": "The unit of measure (defaults to 'units' if not specified)"}}, "required": ["base", "height"]}}}

In [3]:
question = data['question']
question

'Find the area of a triangle with a base of 10 units and height of 5 units.'

In [4]:
functions = data['function']
functions

{'name': 'calculate_triangle_area',
 'description': 'Calculate the area of a triangle given its base and height.',
 'parameters': {'type': 'dict',
  'properties': {'base': {'type': 'integer',
    'description': 'The base of the triangle.'},
   'height': {'type': 'integer', 'description': 'The height of the triangle.'},
   'unit': {'type': 'string',
    'description': "The unit of measure (defaults to 'units' if not specified)"}},
  'required': ['base', 'height']}}

In [5]:
# Code copied from :
# https://github.com/ShishirPatil/gorilla/blob/1d8d51d0d091c33e730d38745745005c9bc7dfc0/berkeley-function-call-leaderboard/model_handler/constant.py#L13
# https://github.com/ShishirPatil/gorilla/blob/1d8d51d0d091c33e730d38745745005c9bc7dfc0/berkeley-function-call-leaderboard/model_handler/utils.py#L9

GORILLA_TO_OPENAPI = {
    "integer": "integer",
    "number": "number",
    "float": "number",
    "string": "string",
    "boolean": "boolean",
    "bool": "boolean",
    "array": "array",
    "list": "array",
    "dict": "object",
    "object": "object",
    "tuple": "array",
    "any": "string",
    "byte": "integer",
    "short": "integer",
    "long": "integer",
    "double": "number",
    "char": "string",
    "ArrayList": "array",
    "Array": "array",
    "HashMap": "object",
    "Hashtable": "object",
    "Queue": "array",
    "Stack": "array",
    "Any": "string",
    "String": "string",
    "Bigint": "integer",
}

def _cast_to_openai_type(properties, mapping, test_category):
    for key, value in properties.items():
        if "type" not in value:
            properties[key]["type"] = "string"
        else:
            var_type = value["type"]
            if mapping == GORILLA_TO_OPENAPI and var_type == "float":
                properties[key]["format"] = "float"
                properties[key]["description"] += " This is a float type value."
            if var_type in mapping:
                properties[key]["type"] = mapping[var_type]
            else:
                properties[key]["type"] = "string"

        # Currently support:
        # - list of any
        # - list of list of any
        # - list of dict
        # - list of list of dict
        # - dict of any

        if properties[key]["type"] == "array" or properties[key]["type"] == "object":
            if "properties" in properties[key]:
                properties[key]["properties"] = _cast_to_openai_type(
                    properties[key]["properties"], mapping, test_category
                )
            elif "items" in properties[key]:
                properties[key]["items"]["type"] = mapping[
                    properties[key]["items"]["type"]
                ]
                if (
                    properties[key]["items"]["type"] == "array"
                    and "items" in properties[key]["items"]
                ):
                    properties[key]["items"]["items"]["type"] = mapping[
                        properties[key]["items"]["items"]["type"]
                    ]
                elif (
                    properties[key]["items"]["type"] == "object"
                    and "properties" in properties[key]["items"]
                ):
                    properties[key]["items"]["properties"] = _cast_to_openai_type(
                        properties[key]["items"]["properties"], mapping, test_category
                    )
    return properties

In [6]:
schema = json.dumps({
            "title": functions["name"],
            "type": "object",
            "description": functions["description"],
            "properties": _cast_to_openai_type(functions["parameters"]["properties"], GORILLA_TO_OPENAPI, "simple"),
            "required": functions["parameters"]["required"]
        }, indent=2)

print(schema)

{
  "title": "calculate_triangle_area",
  "type": "object",
  "description": "Calculate the area of a triangle given its base and height.",
  "properties": {
    "base": {
      "type": "integer",
      "description": "The base of the triangle."
    },
    "height": {
      "type": "integer",
      "description": "The height of the triangle."
    },
    "unit": {
      "type": "string",
      "description": "The unit of measure (defaults to 'units' if not specified)"
    }
  },
  "required": [
    "base",
    "height"
  ]
}


In [7]:
# schema to json file
with open('schema.json', 'w') as f:
    f.write(schema)

In [8]:
!datamodel-codegen  --input schema.json --input-file-type jsonschema --output model.py

In [11]:
pydantic_models_dir = Path(".").resolve() / "pydantic_models"
pydantic_models_dir.mkdir(exist_ok=True, parents=True)

In [14]:



def import_model_from_path(file_path: Path):
    # Ensure the directory of the file is in sys.path
    module_dir = file_path.parent
    if str(module_dir) not in sys.path:
        sys.path.append(str(module_dir))

    # Create a module spec from the file location
    spec = importlib.util.spec_from_file_location(file_path.stem, file_path)
    if spec is None:
        raise ImportError(f"Could not load spec for module at {file_path}")

    # Load the module from spec
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    attributes = [a for a in dir(module) if not a.startswith("__")]
    print(attributes)

    # Access the Model object
    if hasattr(module, "Model"):
        return getattr(module, "Model")
    else:
        raise AttributeError(
            f"Module at {file_path} does not have an object named 'Model'"
        )


Model = import_model_from_path(pydantic_models_dir / "model.py")

['BaseModel', 'CalculateTriangleArea', 'Field', 'Optional', 'annotations']


AttributeError: Module at C:\Users\Alonso\Dropbox\personal\repos\guided-generation-benchmark\evals\bfcl\notebooks\pydantic_models\model.py does not have an object named 'Model'